In [ ]:
%matplotlib inline

In [ ]:
import MDAnalysis as mda
import ARPDF, utils, search_boxes
import importlib
import matplotlib.pyplot as plt
import torch
importlib.reload(search_boxes)
importlib.reload(ARPDF)
importlib.reload(utils)
from ARPDF import *
from utils import *
from search_boxes import StructureSearcher
from ccl4_modifier import CCL4Modifier_C_CL, select_cl_atoms

In [ ]:
u1, u2, modified_atoms, polar_axis = load_structure_data('data/CCl4')
X, Y = generate_grids(10, 256)
filter_fourier = lambda kX, kY, xp: xp.ones_like(kX)
# filter_fourier = lambda kX, kY, xp: (1 - xp.exp(-(kX**2 / 0.3 + kY**2 / 0.1)))**3 * xp.exp(-0.08 * (kX**2 + kY**2))
print(modified_atoms)
sigma0 = 0.2
neg = False
print(f"sigma0 = {sigma0}, neg = {neg}")

In [ ]:
ARPDF_ref = compute_ARPDF(u1, u2, N=512, cutoff=13.0, sigma0=sigma0, grids_XY=(X, Y), modified_atoms=modified_atoms, 
                        polar_axis=polar_axis, periodic=True, filter_fourier=filter_fourier, verbose=True)

In [ ]:
def workflow_for_opt(
        X, Y, ARPDF_ref, 
        filter_fourier=None, 
        sigma0=0.2, 
        exp_name: str="exp", 
        metric: str="1D_average", 
        weight_cutoff=5.0, 
        stretch_distances: List[float] = None,
        neg=False
    ):
    """Demo workflow for structure search and analysis"""
    # Clean and load structure
    clean_gro_box('data/CCl4/CCl4.gro', 'data/CCl4/CCl4_clean.gro')
    universe = mda.Universe('data/CCl4/CCl4_clean.gro')
    if neg:
        ARPDF_ref = ARPDF_ref.copy()
        ARPDF_ref[ARPDF_ref > 0] = 0
    
    # Initialize searcher with CCl4-specific components
    searcher = StructureSearcher(
        output_dir=f"tmp/{exp_name}",
        universe=universe,
        grids_XY=(X, Y),
        ARPDF_ref=ARPDF_ref,
        molecule_selector=select_cl_atoms,
        structure_modifier=CCL4Modifier_C_CL(universe, stretch_distances, periodic=True),
        filter_fourier=filter_fourier,
        sigma0=sigma0,
        metric=metric,
        weight_cutoff=weight_cutoff,
        neg=neg
    )
    
    # Search and save results
    searcher.search()
    searcher.save_results()

In [ ]:
workflow_for_opt(
    X, Y, ARPDF_ref, 
    filter_fourier, 
    sigma0, 
    exp_name="exp_opt_1D_avg", 
    metric="1D_average", 
    stretch_distances=[round(1.0 + 0.1 * i, 1) for i in range(15)], 
    weight_cutoff=5.0, 
    neg=neg
)

In [ ]:
workflow_for_opt(
    X, Y, ARPDF_ref, 
    filter_fourier, 
    sigma0, 
    exp_name="exp_opt_cosine", 
    metric="cosine", 
    stretch_distances=[round(1.0 + 0.1 * i, 1) for i in range(15)], 
    weight_cutoff=5.0, 
    neg=neg
)